In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
#arquitectura neuronal
class HaloToGalaxyModel(nn.Module):
    def __init__(self, input_size=4, output_size=50, hidden_dim=64):
        super(HaloToGalaxyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_size)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  
        return F.softmax(x, dim=1)

# Función para cargar datos desde un CSV
def load_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    X = data.iloc[:, 5:10].values  
    #y = data.iloc[:, 12:16].values  
    y = data.iloc[:, 12].values  #esto sólo carga la masa, la gracia es que para cada modelo usar un única columa
    return X, y

In [3]:
#cargar datos
file_path = 'datasetcompleto.csv'  
X, y = load_data_from_csv(file_path)

In [4]:
k = 50 #división de bins, 50 es lo que dice el paper
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
#transformar etiquetas a bins para problema de clasificacion
bins = np.linspace(y.min(), y.max(), k + 1)
y_binned = np.digitize(y, bins) - 1
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long) 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

hidden_dim = 64 #tamaño de las capas ocultas

model = HaloToGalaxyModel(X.shape[1], k, hidden_dim)
criterion = nn.CrossEntropyLoss()  # Función de pérdida de entropía cruzada
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [12]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  
    optimizer.zero_grad() 
    outputs = model(X_train) 
    loss = criterion(outputs, y_train)      
    loss.backward() 
    optimizer.step() 

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'halo_to_galaxy_model.pth')


50
50
50
50
50
50
50
50
50
50
Epoch [10/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [20/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [30/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [40/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [50/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [60/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [70/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [80/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [90/100], Loss: 3.4163
50
50
50
50
50
50
50
50
50
50
Epoch [100/100], Loss: 3.4163


In [8]:
model.eval()  
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / y_test.size(0)
    print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 52.88%
